<a href="https://colab.research.google.com/github/profteachkids/CHE2064_Spring2022/blob/main/AdiabaticFlash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone --depth 1 https://github.com/profteachkids/chetools.git &> /dev/null
import sys
sys.path.insert(1, "/content/chetools/tools") #Path to CHE module imports

In [ ]:
import che
from collections import namedtuple
from copy import deepcopy
import numpy as np
from scipy.special import expit,logit
from scipy.optimize import root

In [ ]:
class dot_dict(dict):
    __getattr__= dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [ ]:
class Unk():
    pass

class Range(Unk):
    def __init__(self,value,lo,hi):
        self.lo=lo
        self.hi=hi
        self.diff = hi-lo
        self.x=value
        self.shape=(1,)
        self.size = 1

    def flatten(self):
        return np.ravel(logit((self.x-self.lo)/self.diff))

    def unflatten(self,xx):
        return expit(xx.reshape(self.shape))*self.diff +self.lo


class RangeArray(Range):
    def __init__(self,value,lo,hi):
        self.lo=np.atleast_1d(lo)
        self.hi=np.atleast_1d(hi)
        self.diff = self.hi-self.lo
        value = np.atleast_1d(value)
        self.x = value
        self.shape = value.shape
        self.size = value.size

class Comp(Unk):
    def __init__(self,value):
        self.x=np.asarray(value).reshape(-1)
        self.size=self.x.size - 1

    def __repr__(self):
        return f'{self.x}'

    def flatten(self):
        return np.log(self.x[:-1]) + np.log(1.+ (1. - self.x[-1])/self.x[-1])


    def unflatten(self,xx):
        xm1 = np.exp(xx)/(1+np.sum(np.exp(xx)))
        return np.concatenate((xm1, np.atleast_1d(1.-np.sum(xm1))))



In [ ]:
Unk_Tuple = namedtuple('Unk_Tuple', ['keys', 'start', 'end', 'unk'])

def dtox(d):
    d2=dot_dict()
    size=0
    unks={}

    for k,v in d.items():
        idv = id(v)
        if idv in unks:
            unks[idv].keys.append(k)
            continue
        if isinstance(v,Unk):
            unks[idv]=Unk_Tuple([k], size, size+v.size, v)
            size+=v.size
        else:
            d2[k]=v
    x = np.zeros(size)
    for k,v in unks.items():
        x[v.start:v.end]= v.unk.flatten()
            
    def xtod(x,d2):
        for k,v in unks.items():
            unflattened = v.unk.unflatten(x[v.start:v.end])
            for key in v.keys:
                d2[key]=unflattened
        return d2
        
    return x, xtod(x,d2), xtod


In [ ]:
p = che.Props(['Ethanol','Isopropanol', 'Water'])

In [ ]:
# Static parameters (Total feed, feed mole fractions, feed temperature and )
c=dot_dict()
c.Ftot=10 # Total Feed moles
c.Fz = np.array([1/3, 1/3, 1/3]) # Equimolar feed composition
c.FT = 450 # Feed temperature
c.flashP= 101325 # Flash drum pressure

c.Vy = Comp(c.Fz) # Guess vapor/liquid composition equal to feed
c.Lx = Comp(c.Fz) # Comp - constrains mole fractions to behave like mole fractions!
c.flashT = Range(360, 273.15, c.FT)  # Guess and bounds for flash temperature
c.Vtot = Range(c.Ftot/2, 0., c.Ftot)  # Guess half of feed in vapor
c.Ltot = Range(c.Ftot/2, 0., c.Ftot)

In [ ]:
x, d, xtod = dtox(c)

In [ ]:
def eqs(x,d):
    d = xtod(x, d)    
    V = d.Vy * d.Vtot # Moles of each component = mole fractions * total moles
    L = d.Lx * d.Ltot
    F = d.Fz * d.Ftot
    mass_balance = F - V - L # Mass balance for each component (vectors!)

    # Hmix calculates the enthalpy given the temperature and moles of each
    # component in the vapor and liquid phases
    FH = p.Hl(nL=F, T=d.FT)
    VH = p.Hv(nV=V, T=d.flashT)
    LH = p.Hl(nL=L, T=d.flashT)
    energy_balance = (FH - VH - LH)

    # Raoults with NRTL activity coefficient correction.  One-liner!
    fugL = d.Lx  * p.NRTL_gamma(d.Lx,d.flashT)* p.Pvap(d.flashT)
    fugV = d.Vy * d.flashP
    VLE = fugL - fugV
    return np.concatenate([mass_balance, np.atleast_1d(energy_balance), VLE])


In [ ]:
sol=root(lambda x: eqs(x,d), x).x

In [ ]:
xtod(sol,d)

{'FT': 450,
 'Ftot': 10,
 'Fz': array([0.33333333, 0.33333333, 0.33333333]),
 'Ltot': array([6.02836122]),
 'Lx': array([0.32122647, 0.32919072, 0.34958281]),
 'Vtot': array([3.97163878]),
 'Vy': array([0.35170977, 0.33962121, 0.30866903]),
 'flashP': 101325,
 'flashT': array([352.85497499])}